# 📖 MACD (Moving Average Convergence Divergence)

---

## 📌 Definición

El **MACD** (*Moving Average Convergence Divergence*) es un **indicador técnico de momentum** desarrollado por Gerald Appel en 1979.  
Mide la **diferencia entre dos medias móviles exponenciales (EMA)** de distintos periodos para identificar cambios en la fuerza, dirección y duración de una tendencia.

---

## 📐 Fórmulas

### 1. Línea MACD
Se define como la diferencia entre una **EMA rápida** y una **EMA lenta**:

$$
MACD_t = EMA_{\text{rápida}}(t) - EMA_{\text{lenta}}(t)
$$

Generalmente se usan periodos de 12 y 26 días:

$$
MACD_t = EMA_{12}(t) - EMA_{26}(t)
$$

---

### 2. Línea de Señal
Es una **EMA suavizada** del MACD, que sirve como referencia para generar señales de compra o venta:

$$
\text{Señal}_t = EMA_{9}(MACD_t)
$$

---

### 3. Histograma MACD
Representa la diferencia entre la línea MACD y la línea de señal:

$$
Histograma_t = MACD_t - \text{Señal}_t
$$

---

## 📊 Interpretación

- **Cruce alcista**: Cuando $MACD_t$ cruza **por encima** de la línea de señal → posible señal de compra.  
- **Cruce bajista**: Cuando $MACD_t$ cruza **por debajo** de la línea de señal → posible señal de venta.  
- **Histograma positivo creciente**: Señal de fortalecimiento de la tendencia alcista.  
- **Histograma negativo decreciente**: Señal de fortalecimiento de la tendencia bajista.  
- **Divergencias**: Cuando el precio y el MACD muestran direcciones opuestas, puede anticipar un cambio de tendencia.

---

## 📌 Ventajas
- Combina conceptos de **tendencia** y **momentum**.  
- Fácil de interpretar visualmente con líneas y barras.  
- Funciona en diferentes marcos temporales.  

---

## ⚠️ Limitaciones
- Genera señales falsas en **mercados laterales** (rangos sin tendencia).  
- Requiere confirmación con otros indicadores o soportes/resistencias.  
- Es un **indicador rezagado** al basarse en medias móviles.

---

## 📚 Referencias
- Appel, G. (2005). *Technical Analysis: Power Tools for Active Investors*.  
- Murphy, J. J. (1999). *Technical Analysis of the Financial Markets*.  
- Investopedia: [MACD Indicator](https://www.investopedia.com/terms/m/macd.asp)

---

In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

In [2]:
#@title Seleccione Acción y Fecha {run:'auto'}
ticker_symbol = "NVDA" #@param ["AAPL", "MSFT", "NVDA", "GOOGL", "AMZN", "META", "TSM","BRK.B", "V", "JPM", "XOM", "LLY", "MRK", "UNH", "PG", "MA","CVX", "KO", "PEP", "COST", "TMO", "ORCL", "CSCO", "NKE", "VZ", "ASML", "TXN", "ABT", "TM", "SAP", "AMD", "NFLX", "NOW", "ADBE", "LVMUY", "BABA", "SHEL", "TMUS", "QCOM", "PFE", "SNY", "AZN", "TOT", "GSK", "RIO", "BHP", "MCD​"]
#ticker = "BTC-USD" #@param ["BTC-USD", "ETH-USD", "USDT-USD", "XRP-USD", "LTC-USD", "ADA-USD", "DOT-USD", "BCH-USD", "XLM-USD", "LINK-USD"]

start_date = '2024-09-09' #@param {type:'date'}
end_date = '2025-09-09' #@param {type:'date'}

In [3]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_candlestick(ticker, start_date, end_date):
    """
    Descarga datos históricos, grafica un candlestick interactivo con Plotly,
    y muestra el indicador MACD.
    """
    try:
        data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)

        # droplabel level 1
        data.columns = data.columns.droplevel(1)

        if data.empty:
            print(f"No se encontraron datos para {ticker} entre {start_date} y {end_date}")
            return

        # Calcular MACD y Señal
        data['EMA_12'] = data['Close'].ewm(span=12, adjust=False).mean()
        data['EMA_26'] = data['Close'].ewm(span=26, adjust=False).mean()
        data['MACD'] = data['EMA_12'] - data['EMA_26']
        data['Signal'] = data['MACD'].ewm(span=9, adjust=False).mean()

        # Crear subplots: 1 fila, 2 filas (candlestick + MACD)
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                            vertical_spacing=0.05,
                            row_heights=[0.7, 0.3],
                            subplot_titles=(f"{ticker} - Velas Japonesas", "MACD"))

        # Gráfico Candlestick
        fig.add_trace(
            go.Candlestick(
                x=data.index,
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'],
                name='OHLC',
                increasing_line_color='green',
                decreasing_line_color='red'
            ),
            row=1, col=1
        )

        # Línea MACD
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=data['MACD'],
                line=dict(color='blue', width=1.5),
                name='MACD_t(EMA12(t)-EMA26(t))'
            ),
            row=2, col=1
        )

        # Línea Señal
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=data['Signal'],
                line=dict(color='orange', width=1),
                name='Línea Señal_t (EMA9(MACD_t))'
            ),
            row=2, col=1
        )

        # Layout general
        fig.update_layout(
            title=f"{ticker} | Candlestick con MACD ({start_date} a {end_date})",
            xaxis_rangeslider_visible=False,
            template="plotly_white",
            height=800,
            margin=dict(t=60, b=40),
            legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
        )

        # Etiquetas de ejes
        fig.update_yaxes(title_text="Precio", row=1, col=1)
        fig.update_yaxes(title_text="MACD", row=2, col=1)

        fig.show()

    except Exception as e:
        print(f"Ocurrió un error: {e}")

In [4]:
plot_candlestick(ticker_symbol, start_date, end_date)

[*********************100%***********************]  1 of 1 completed


In [5]:
# Compra-Venta 1
inversion = 10000
compra = 200.77
venta = 213.80
no_de_acciones = inversion / compra
ganancia = (no_de_acciones * venta) - inversion
print(f"Inverti ${inversion:,.2f} y tuve una ganancia de ${ganancia:,.2f} que es un {ganancia/inversion * 100:,.2f}%")

Inverti $10,000.00 y tuve una ganancia de $649.00 que es un 6.49%


In [6]:
# Compra-Venta 2
inversion = 10000
compra = 213.00
venta = 230.89
no_de_acciones = inversion / compra
ganancia = (no_de_acciones * venta) - inversion
print(f"Inverti ${inversion:,.2f} y tuve una ganancia de ${ganancia:,.2f} que es un {ganancia/inversion * 100:,.2f}%")

Inverti $10,000.00 y tuve una ganancia de $839.91 que es un 8.40%
